#### [**Mathematical Modeling of Unsteady Inviscid Flows**](https://www.springer.com/gp/book/9783030183189)
**by Jeff D. Eldredge** (Springer, 2019)

This Jupyter notebook and associated code serve as a companion to the book. The notebook is powered by the [`PotentialFlow`](https://github.com/darwindarak/PotentialFlow.jl) package, written in the [Julia language](https://julialang.org/) by Darwin Darakananda and Jeff D. Eldredge. *The code is released under the [MIT license](https://opensource.org/licenses/MIT).*
<hr />

<!--NAVIGATION-->
< [Streamfunction of rigid body motion](3.1-StreamfunctionOfRigidBody.ipynb) | [Contents](Index.ipynb) | [Corner and wedge flows](3.3-CornerWedge.ipynb) >

<a id='top'></a>
## Basic potential flows in the plane

Here, we explore basic potential flows in the two-dimensional plane, and particularly, the flow fields associated with them. This notebook supports **Section 3.2.2** in the book.

#### First, carry out the preparatory steps.

Load the necessary packages.

In [ ]:
using PotentialFlow
using Plots
pyplot()

Set up the output grid for evaluating fields. We will also set up a 2d array of complex coordinates, since we will be evaluating complex potentials and velocities later. Note that the `Z` array below, like other 2d arrays, is indexed first with the `x` location and second with the `y` location. This means each column of the array is associated with a row in the grid, and vice versa.

In [ ]:
nx = 201; ny = 201
xmin = -2; xmax = 2
ymin = -2; ymax = 2
x = range(xmin,xmax,length=nx)
y = range(ymin,ymax,length=ny)

# Output grid of complex coordinates.
Z = [xi + im*yi for xi in x, yi in y]

# Initialize complex potential.
F = zeros(ComplexF64,nx,ny);

<a id='freestream'></a>
#### Uniform flow.

Let's start with a uniform flow. In the previous notebook on the [streamfunction of rigid-body motion](3.1-StreamfunctionOfRigidBody.ipynb) we saw an example of streamlines associated with a moving rigid body. A uniform flow is closely related with rigid-body translation. But here we will evaluate the complex potential of this uniform flow.

Set the uniform flow. Note that the complex velocity is in its usual conjugate form. **Explore other values!**

In [ ]:
U∞ = 1.0
V∞ = -2.0
W∞ = U∞-im*V∞

Evaluate the complex potential field (3.86).

In [ ]:
F .= W∞*Z;

Plot the streamlines. Here, we make use of the fact that the streamfunction is the imaginary part of the complex potential. We need to transpose the array, as we did in the last notebook.

In [ ]:
contour(x,y,imag.(F)',ratio=1,legend=:false,color=:black,xlabel="x",ylabel="y",
    xlim=(xmin,xmax),ylim=(ymin,ymax))

In the `PotentialFlow` package we can create this uniform flow field with the `Freestream` function. We supply this function with the complex velocity given in non-conjugate form:

In [ ]:
fs = Freestreams.Freestream(U∞+im*V∞)

And we can plot the streamlines with the `streamlines` function. Note that this function (technically, a *plot recipe* for the `Plots` package) expects a *tuple* of potential flow elements. Here, we supply it with only one such element: the uniform flow. In Julia, a tuple with only a single entry is assigned by `(entry1,)`.

In [ ]:
streamlines(x,y,(fs,),ratio = 1, legend = :false,
    xlabel="x",ylabel="y",xlim=(xmin,xmax),ylim=(ymin,ymax))

We can obtain the velocity of this flow (or any other potential flow) in the `PotentialFlow` package at an evaluation point `z` by using the `induce_velocity` function:

In [ ]:
# evaluation point
z = 1.0+0.2im

# time of evaluation (irrelevant, but still needed as an argument)
t = 0.0

induce_velocity(z,fs,t)

As expected, the function simply returns the uniform freestream velocity.

[Return to top of notebook](#top)

<a id='point-vortex'></a>
#### Point vortex.

Now let's create a single point vortex and inspect the flow field associated with it. Here we will evaluate equation (3.88).

First we will place this at the origin, with strength 1.

In [ ]:
Γ = 1.0;

Evaluate it.

In [ ]:
F .= Γ/(2π*im)*log.(Z);

ane plot its streamlines

In [ ]:
contour(x,y,imag.(F)',ratio=1,legend=:false,color=:black,xlabel="x",ylabel="y",
    xlim=(xmin,xmax),ylim=(ymin,ymax))

Now let us place the vortex at another location, $(x,y) = (1,1)$. 

In [ ]:
zv = 1+im*1

Evaluate the complex potential now with the vortex at the new location.

In [ ]:
F .= Γ/(2π*im)*log.(Z .- zv);

and plot it

In [ ]:
contour(x,y,imag.(F)',ratio=1,legend=:false,color=:black,xlabel="x",ylabel="y",
    xlim=(xmin,xmax),ylim=(ymin,ymax))

The flow pattern is the same (concentric circles), but the center has shifted!

With the `PotentialFlow` package, it is easy to generate this with the function `Vortex.Point`:

In [ ]:
v = Vortex.Point(zv,Γ)

and plot the streamlines, once again, with the `streamlines` recipe:

In [ ]:
streamlines(x,y,(v,),ratio = 1, legend = :false,
    xlabel="x",ylabel="y",xlim=(xmin,xmax),ylim=(ymin,ymax))

Once again, we can evaluate the velocity at any location with the `induce_velocity` function

In [ ]:
z = 0.0+0.0im
t = 0.0 # time (irrelevant, again)

induce_velocity(z,v,t)

This is a good opportunity to make an observation about the scalar potential field, $\varphi$, of a point vortex. Let us evaluate the complex potential associated with a vortex at two points, one just above the negative $x$ axis extending from the vortex center, and one just below it:

In [ ]:
z₊ = -1+1.00000001im # above the line
z₋ = -1+0.99999999im; # below the line

complexpotential([z₊ z₋],v)

Note the jump in the real part. In fact, it should be noted that the jump in this real part is **exactly equal to the strength of the vortex**. This jump is the *branch cut* in the log function. In the case of a vortex, the consequence of this branch cut is that the strength of the vortex is equal to the jump in scalar potential across the branch cut:

$$\Gamma = \varphi_{+} - \varphi_{-}$$

Why did we know to look along the negative $x$ axis from the center of the vortex? Julia, like most mathematical software, places the branch cut of the log along the $-x$ axis (in a coordinate system based at the origin of the log). Further aspects of multi-valuedness for planar singularities is discussed in detail in **Note 3.2.2**.

In spite of the branch cut in the scalar potential, the velocity is single valued. We can confirm this by evaluating the velocity at the same two locations, above and below the branch cut:

In [ ]:
t = 0.0
induce_velocity([z₊ z₋],v,t)

It is clear that the velocity is continuous. (The small difference is due to the finite distance between our two evaluation points.)

[Return to top of notebook](#top)

<a id='source-sink'></a>
#### Source and sink.

A point (or monopole) source is similar in structure to a vortex, but has streamlines that radiate outward from the center. A sink (a source with negative strength) has streamlines that radiate inward. The complex potential is given by (3.88) again.

Let's place a source of unit strength at $(1,1)$.

In [ ]:
zs = 1.0+im*1.0
Q = 1.0;

Use the `PotentialFlow` package's `Source.Point` function to create the element

In [ ]:
s = Source.Point(zs,Q)

and plot the streamlines

In [ ]:
streamlines(x,y,(s,),ratio = 1, legend = :false,
    xlabel="x",ylabel="y",xlim=(xmin,xmax),ylim=(ymin,ymax))

Note the presence of the branch cut in the streamfunction field! The streamfunction of a source or sink, equation (3.77), is multi-valued. We can see this by once again inspecting the potential: 

In [ ]:
z₊ = -1+1.00000001im # above the cut
z₋ = -1+0.99999999im; # below the cut

complexpotential([z₊ z₋],s)

Note the jump in the imaginary part. Analogous to the case of the vortex, the jump in this imaginary part is **exactly equal to the strength of the source**. This is because this jump represents the difference between the streamfunction values on two bounding streamlines of the flow. These streamlines happen to be right next to each other, but nevertheless, the flow between them fans out from the source. In general, as discussed on page 27 in the book, *the difference in the streamfunction values on two streamlines is equal to the volume flow rate (per unit depth into the screen) of the flow between the streamlines*. That is exactly what the strength $Q$ represents! Try changing `Q` above and observing the corresponding change in the jump.

$$Q = ψ_{+} - ψ_{-}$$

As with the vortex, the velocity field around the source is continuous.

[Return to top of notebook](#top)

<a id='doublet'></a>
#### Doublet (or dipole).

A doublet is characterized by a position and a vector strength. The strength vector specifies the magnitude and the orientation of the doublet. Its complex potential is given by equation (3.90), with strength $a_1$. Note that the complex strength vector defines the *axis* of the doublet. Along this axis, the **flow proceeds *opposite* the direction in which the strength vector $a_1$ points**.

The scalar potential is written in vector form in (3.79), and the complex and vector strengths are related in (3.91).

In `PotentialFlow` we can create a doublet by specifying its position and complex strength. However, note that the strength $S$ we specify for this function is equivalent to **half of the complex strength $a_1$** defined in (3.90). 

In [ ]:
# Position
zd = 0.0+0.0im

# Strength.
S = 1.0+0im

# Set up the doublet element.
d = Doublets.Doublet(zd,S)

In [ ]:
streamlines(x,y,(d,),ratio = 1, legend = :false,
    xlabel="x",ylabel="y",xlim=(xmin,xmax),ylim=(ymin,ymax),
    levels=range(-1,1,length=21))

Note that the streamlines of a doublet are circles, all tangent to the $x$ axis.

Also, note that doublets (and all other higher-order singularities) have single-valued potentials and streamfunctions. Only the fundamental singularities (vortex and source) have branch cuts. In the next notebook on [corner and wedge flows](3.3-CornerWedge.ipynb), we will see branch cuts occur in another potential flow context.

[Return to top of notebook](#top)

<a id='combos'></a>
### Combination of potential flows.

<a id='kelvin'></a>
#### A Kelvin oval

Let's combine the preceding flows. We will construct a *Kelvin oval*, consisting of two counter-rotating point vortices arranged along the $y$ axis (at $-i h$ with strength $\Gamma$ and $ih$ with strength $-\Gamma$) and a uniform flow $U_\infty$ in the $x$ direction that opposes the flow that this vortex pair generates along the centerline.

The complex velocity field of this flow is given by

$w(z) = U_\infty + \dfrac{\Gamma}{2\pi i}\left( \dfrac{1}{z+ih} - \dfrac{1}{z-ih}\right)$

In [ ]:
U∞ = 1.0
Γ = 5.0
h = 1.0
fs = Freestreams.Freestream(U∞)
vlo = Vortex.Point(-im*h,Γ)
vhi = Vortex.Point(im*h,-Γ)
sys = (fs,vlo,vhi)

In [ ]:
streamlines(x,y,sys,ratio = 1, legend = :false,
    xlabel="x",ylabel="y",xlim=(xmin,xmax),ylim=(ymin,ymax))

The *stagnation streamline* exhibits an oval shape around the vortex pair. It is a good exercise to show that the stagnation points generated by this flow occur at

$$x_s = \pm \left[ \dfrac{\Gamma h}{\pi U_\infty} - h^2\right]^{1/2}$$

Note that the strength of the vortices must be at least $\pi U_\infty h$ to generate a closed body. Evaluate the position of the stagnation points:

In [ ]:
xs = sqrt(Γ*h/(π*U∞)-h^2)

Check that the velocity is indeed zero at this location, using the `induce_velocity` function of the `PotentialFlow` package. (The last argument of this function is time, which is irrelevant in this steady flow.)

In [ ]:
induce_velocity(xs+0im,sys,0.0)

[Return to top of notebook](#top)

<a id='doublet-freestream'></a>
#### Doublet in a uniform flow

Let us combine a doublet at the origin, whose axis points along the $-x$ axis, with a uniform flow in the $x$ direction.

In [ ]:
zd = 0.0+0.0im
S = 1.0+0.0im
U∞ = 1.0;

Create the doublet-freestream system

In [ ]:
sys = (Doublets.Doublet(zd,S),Freestreams.Freestream(U∞))

and plot its streamlines

In [ ]:
streamlines(x,y,sys,ratio = 1, legend = :false,
    xlabel="x",ylabel="y",xlim=(xmin,xmax),ylim=(ymin,ymax),
    levels=range(-2,2,length=21))

Now we've generated a closed body that is a perfect circle! What is the radius of the circle? Using the complex velocities from (3.86) and (3.90), you can show that the stagnation points are at

$$x_s = \pm \left( \dfrac{a_1}{2 \pi U_\infty}\right)^{1/2} $$

Since it is a circular stagnation streamline, this must be the radius of the circle. Let's calculate that radius here (remembering that our strength $S$ is equivalent to $a_1/2$):

In [ ]:
R = sqrt(real(S)/(π*U∞))

[Return to top of notebook](#top)

#### Computing pressure

In these inviscid flows, incompressible and irrotational (almost everywhere), the Bernoulli equation (3.60) is valid

$$p + \frac{1}{2}\rho |\mathbf{v}|^2 + \rho\dfrac{\partial \varphi}{\partial t} = B(t)$$

where $\varphi$ is the scalar potential field and $B(t)$ is the uniform *Bernoulli constant*. Let us compute the pressure distribution around the closed circular streamline that we generted in the previous example.

First, we set up an array of points on the circle. We just computed the radius of the circle:

In [ ]:
# angular positions
θ = range(0,2π,length=201)

# complex coordinates of these positions
z = R*exp.(im*θ);

This is a steady flow, so we need not worry about $\partial\varphi/\partial t$. Since all points in the plane share the same value of $B$, we can refer the points on the circular streamline to a point at infinity, where

$$p_{\infty} + \frac{1}{2} \rho U_\infty^2 = B(t)$$

From this comparison, we can calculate the pressure coefficient:

$$C_p = \dfrac{p-p_\infty}{\rho U_\infty^2/2}$$

In this case, this is

$$C_p(\theta) = \dfrac{\rho (U_\infty^2-|\mathbf{v}|^2)/2}{\rho U_\infty^2/2} = 1 - |\mathbf{v}|^2/U_\infty^2$$

In [ ]:
t = 0.0
Cp = 1 .- abs.(induce_velocity(z,sys,t)).^2/U∞^2;

Plot this pressure coefficient as a function of angular position:

In [ ]:
plot(θ,Cp,legend=:false,xlims=(0,2π),ylims=(-3,3),xlabel="θ",ylabel="Cp(θ)")

Note that $\theta = 0$ is at the rear of the circle (at $x = R$), $\theta = \pi/2$ is at the top $y = R$, and $\theta = \pi$ is at the front, $x = -R$). This distribution is symmetric in both the $x$ and $y$ directions, implying that it exerts **no net force** on the circle. This is a specific example of *d'Alembert's paradox*.

<!--NAVIGATION-->
< [Streamfunction of rigid body motion](3.1-StreamfunctionOfRigidBody.ipynb) | [Contents](Index.ipynb) | [Corner and wedge flows](3.3-CornerWedge.ipynb) >